# Purpose
This cadCAD model is a reimplementation of this vanilla python vault model:
https://github.com/marek-new/toy_vault_model
(more documentation there including partial math spec)

# Imports and dependencies

In [1]:
# cadCAD standard dependencies

# cadCAD global simulation configuration list
from cadCAD import configs

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

#other imports
from random import random
from random import randint
# For analytics
import pandas as pd
# For visualization
import plotly.express as px


# Define State Variables
information we need to track throughout the simulation

In [2]:
#these are the variables we care about e.g. "maximize TVL / rewardRate"
initial_state = {
    # vault balance aka TVL
    'vault_balance': [0.0],
    #staker balances
    'staker_balances': [3.0, 5.0, 8.0, 7.5],
    #deposits in the vault
    'staker_deposits': [0.0, 0.0, 0.0, 0.0],
    #accumulated rewards in the vault
    'staker_rewards': [0.0, 0.0, 0.0, 0.0],
    #staker expectations of yield
    'expected_rewards': [0.5, 2.0, 1.5, 1.0],
}
initial_state


{'vault_balance': [0.0],
 'staker_balances': [3.0, 5.0, 8.0, 7.5],
 'staker_deposits': [0.0, 0.0, 0.0, 0.0],
 'staker_rewards': [0.0, 0.0, 0.0, 0.0],
 'expected_rewards': [0.5, 2.0, 1.5, 1.0]}

# Define System Parameters
these parameters correspond to scenarios we wish to investigate e.g. different assumptions
if we wanted to explore the effect of a bunch of different values for rewardRate we would put them in a list here

In [3]:
system_params = {
    'rewardRate': [7.0],
    # if instead we wanted to do an A/B test of rewardRate=7.0 vs. rewardRate=9.0 we would do the follwing:
    # 'rewardRate': [7.0, 9.0]
}

system_params

{'rewardRate': [7.0]}

# Helper functions

In [4]:
def rewardShare(share,total,reward):
    if total == 0.0:
        return 0.0
    return reward*share/total

# Policy Functions - make decisions
but, updating the state will happen elsewhere!

cadCAD has made a design decision that forces the modeler to separate decisions and state updates (this is generally a good pattern)

Policy funcitons have a very specific and regular format in that they accept
params, the current substep, state_history, previous_state

In [5]:
#policy decision for the staker
def p_staker(params, substep, state_history, previous_state):
    #get the information needed to make decisions
    balances = previous_state['staker_balances']
    deposits = previous_state['staker_deposits']
    rewards = previous_state['staker_rewards']
    TVL = previous_state['vault_balance']
    expectations = previous_state['expected_rewards']
    RR = params['rewardRate']
    #the pattern is to compose state update vectors for all the Stakers
    balances_change = [0.0, 0.0, 0.0, 0.0]
    deposits_change = [0.0, 0.0, 0.0, 0.0]
    rewards_change = [0.0, 0.0, 0.0, 0.0]
    
    #make decisions for each staker
    for i in range(0,len(balances)):
        if balances[i] >= 1.0 and random() < 0.5: #deposit
            deposits_change[i] = randint(1, int(balances[i]))
            balances_change[i] = -deposits_change[i]
        elif(rewardShare(deposits[i],TVL,RR) < expectations[i]): #claim and withdraw
            #claim
            balances_change[i] = rewards[i]
            rewards_change[i] = -rewards[i]
            #withdraw all
            balances_change[i] = balances_change[i] + deposits[i]
            deposits_change[i] = -deposits[i]
        elif random() < 0.5 and rewards[i] > 0 and expectations[i] < rewards[i]:
            #claim
            balances_change[i] = rewards[i]
            rewards_change[i] = -rewards[i]
    #the policy functions return one or more state-change vectors that will be used to update state later
    return {'balances_change': balances_change,
            'deposits_change': deposits_change,
            'rewards_change': rewards_change,
            } 


In [6]:
def p_vault(params, substep, state_history, previous_state):
    deposits = previous_state['staker_deposits']
    TVL = previous_state['vault_balance']
    
    RR = params['rewardRate']

    rewards_change = [0.0, 0.0, 0.0, 0.0]
    
    for i in range(0,len(deposits)):
        rewards_change[i] = rewardShare(deposits[i],TVL,RR)

        
    return {'rewards_change': rewards_change}

# State Update Functions
these define how our model state changes over each time step (each iteration through the model)
the state changes were decided upon by the policy functions above
* params is a dictionary containing system params declared above
* substep is an integer corresponding to the partial state update block being executed
* state_history is a list containing all states in all previous timesteps
* previous_state is a dictionary that contains the state in the previous timestep or substep
* policy_input is a dictionary containing changes resulting from policy functions (above)

In [7]:
def s_vault_balance(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    
    print("s_vault_balance policy input:", policy_input)
    variable = 'vault_balance'
    value = previous_state['vault_balance']

    deposits_change = policy_input['deposits_change']
    for i in range(0,len(deposits_change)):
        value = value + deposits_change[i]
    return (variable, value)


In [8]:
def s_staker_balances(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'staker_balances'
    value = previous_state['staker_balances']
    print("s_staker_balances valiue: ", value)
    balances_change = policy_input['balances_change']
    for i in range(0,len(balances_change)):
        value[i] = value[i] + balances_change[i]

    return (variable, value)

    

In [9]:
def s_staker_deposits(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'staker_deposits'
    value = previous_state['staker_deposits']
    deposits_change = policy_input['deposits_change']
    
    for i in range(0,len(deposits_change)):
        value[i] = value[i] + deposits_change[i]

    return (variable, value)


In [10]:
def s_staker_rewards(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'staker_rewards'
    value = previous_state['staker_rewards']
    rewards_change = policy_input['rewards_change']
    
    for i in range(0,len(rewards_change)):
        value[i] = value[i] + rewards_change[i]

    return (variable, value)

# PSUBs - Partial State Update Blocks
define the order of policy functions and state update functions

In [11]:
partial_state_update_blocks = [
    {# corresponds to substep 1
        'policies': { # policy functions in this block are executed in parallel
            'policy_staker': p_staker,
        },
        'variables': { # state update functions that use the above policy (executed in parallel)
            'vault_balance': s_vault_balance,
            'staker_balances': s_staker_balances,
            'staker_deposits': s_staker_deposits,
            'staker_rewards': s_staker_rewards
        }
    },
    {# corresponds to substep 2 - executes sequentially after substep 1
        'policies': { #policy functions in this block are executed in parallel 
            'policy_vault':p_vault,
        },
        'variables': {# state update functions that use the above policy (executed in parallel)
            'staker_rewards': s_staker_rewards
        }
    }
    
]

# Simulation Configuration

In [12]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(10), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})


In [13]:
del configs[:] # Clear any prior configs for ``reasons``

In [14]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

# Execution

In [15]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

In [16]:
simulation = Executor(exec_context=exec_context, configs=experiment.configs)

In [17]:
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 1, 1, 5)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 1, 1, 5)
Execution Method: local_simulations
Execution Mode: single_threaded


TypeError: unsupported operand type(s) for /: 'float' and 'list'

# Output preparation

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result.head(5)


# Analysis

In [ ]:
#clean substeps
first_ind = (simulation_result.substep==0)&(simulation_result.timestep==0)
last_ind = simulation_result.substep == max(simulation_result.substep)
inds_to_drop = (first_ind | last_ind)
df = simulation_result.loc[inds_to_drop].drop(columns=['substep'])

df.head(5)

In [ ]:
px.line(df,
           x='timestep',
           y=['vault_balance'])